## libraries and function 

In [1]:
!pip install impyute
from sklearn import datasets
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as skLDA
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from scipy import stats
import numpy as np
import impyute as impy
from fancyimpute import IterativeSVD, SoftImpute, NuclearNormMinimization
import pandas as pd
import time

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


The function `mle` allows us to compute the MLEs from training data with monotone missing data.

We denote
$$n = \begin{pmatrix}
n_1^{(1)} & n_1^{(2)} &...&n_1^{(K)}\\
\vdots & \vdots &\ddots&\vdots\\
n_G^{(1)} & n_G^{(2)} &...&n_G^{(K)}
\end{pmatrix}$$
$$p = (p_1,p_2,...,p_K)$$


### MLE function 

In [2]:
import numpy as np
def mle(Xtrain, n, p, G):
    '''
    Xtrain: list of input. The ith element of the list contains the sample from
    the ith class.
    '''
    if p[0]==1:
        # the array that contains the means of each block for the 1st block
        mus = [np.mean(Xtrain[g][:,0]) for g in np.arange(G)]
        S = [(n[g,0]-1)*np.var(Xtrain[g][:,0]) for g in np.arange(G)]
    else:
        mus = [np.mean(Xtrain[g][:,0:p[0]], axis = 0) for g  in np.arange(G)]
        S = [(n[g,0]-1)*np.cov(Xtrain[g][:,0:p[0]],rowvar =False) 
             for g in np.arange(G)]
    
    mus = np.asarray(mus).T # so that each column is the mean of a class
    S = sum(S)/(sum(n[:,0])) 
    S = S.reshape((p[0],-1))
    for i in np.arange(1,len(p)):
        W = [(n[g,i]-1)*np.cov(Xtrain[g][0:n[g,i],0:p[i]],
                              rowvar=False) for g in np.arange(G)]
        W = sum(W)
        
        P = np.matmul(W[(p[i-1]):p[i], 0:p[i-1]],
                      np.linalg.inv(W[0:p[i-1],0:p[i-1]]))
        Q = (W[p[i-1]:p[i],p[i-1]:p[i]]-
            np.matmul(P, W[0:p[i-1],p[i-1]:p[i]]))/sum(n[:,i])
        xmeans = [np.mean(Xtrain[g][0:n[g,i],0:p[i]], axis = 0) 
                  for g in np.arange(G)]
        
        xmeans = np.asarray(xmeans)
        xmeans = xmeans.T
        mus = np.vstack((mus, xmeans[p[i-1]:p[i],:]
                       - np.matmul(P, xmeans[0:p[i-1]]-mus)))
        S21 = np.matmul(P, S)
        S = np.vstack((np.hstack((S, S21.T)),
                       np.hstack((S21, Q+np.matmul(P, S21.T)))))
    return [mus, S]

### nan function 


In [3]:
'''
function that create data list that contain missing values
The input X is a numpy array, y is the label
the function return a list where the ith element of 
the list belongs to the ith class
'''

def make_nan_list(X,y,G, n, p):
    # note that the label should go from 0 to G-1
    data = []
    for g in np.arange(G):
        data.append(X[y==g,:])
        for k in np.arange(len(p)-1):
            data[g][n[g,k+1]:n[g,k], p[k]:] = np.nan
    return data


In [4]:
def missing_rate(Xtrain, ytrain, n, p, G):
    # function that compute the missing rate of a given pattern    
    Xtr_nan_list = make_nan_list(Xtrain,ytrain,G, n, p)
    # make NA data
    # since making function changes the order of observation
    # we need to generate new ytr from Xtr_nan    
    Xtr_nan, ytr = Xtr_nan_list[0], np.repeat(0, len(Xtr_nan_list[0]))
    for g in np.arange(1,G):
        Xtr_nan = np.vstack((Xtr_nan, Xtr_nan_list[g]))
        ytr = np.hstack((ytr, np.repeat(g, len(Xtr_nan_list[g]))))

    # percentage of missing values
    per_missing = np.mean(np.isnan(Xtr_nan))
    return per_missing

### compute_err function 

In [5]:
def err(mus, S, mus_est, S_est):
  err_rate = (np.linalg.norm(mus_est-mus))/mus.size 
  err_rate += (np.linalg.norm(S_est-S))/S.size 
  return err_rate

In [6]:
def compute_err_EM(Xtrain, ytrain, n, p, G):      
    # make NAs
    Xtr_nan_list = make_nan_list(Xtrain,ytrain,G, n, p)
    # make NA data
    # since making function changes the order of observation
    # we need to generate new ytr from Xtr_nan    
    Xtr_nan, ytr = Xtr_nan_list[0], np.repeat(0, len(Xtr_nan_list[0]))
    for g in np.arange(1,G):
        Xtr_nan = np.vstack((Xtr_nan, Xtr_nan_list[g]))
        ytr = np.hstack((ytr, np.repeat(g, len(Xtr_nan_list[g]))))

    scaler = StandardScaler()
    scaler.fit(Xtr_nan)
    Xtr_nan = scaler.transform(Xtr_nan)
    Xtrain = scaler.transform(Xtrain)
    for g in range(G):
      Xtr_nan_list[g] = scaler.transform(Xtr_nan_list[g])

    mus = [np.mean(Xtrain[ytrain==g,:], axis=0) for g in np.arange(G)]
    mus = np.asarray(mus) # each row is a mean of a class
    S = [(sum(ytrain==g)-1)*np.cov(Xtrain[ytrain==g,:],rowvar =False) 
             for g in np.arange(G)]
    S = np.asarray(S)/len(ytrain)

    # percentage of missing values
    per_missing = np.mean(np.isnan(Xtr_nan))

    start = time.time()
    Xtr_em = impy.em(Xtr_nan, loops=10)
    mus_em = np.array([np.mean(Xtr_em[ytrain==g,:], axis=0) for g in np.arange(G)])
    S_em = np.array([(sum(ytrain==g)-1)*np.cov(Xtr_em[ytrain==g,:], rowvar =False) 
             for g in np.arange(G)])
    S_em = S_em/len(ytrain)
    em_err = err(mus, S, mus_em, S_em)
    em_time = time.time()-start   

    return em_err, em_time, per_missing

In [7]:
def compute_err_MICE(Xtrain, ytrain, n, p, G):      
    # make NAs
    Xtr_nan_list = make_nan_list(Xtrain,ytrain,G, n, p)
    # make NA data
    # since making function changes the order of observation
    # we need to generate new ytr from Xtr_nan    
    Xtr_nan, ytr = Xtr_nan_list[0], np.repeat(0, len(Xtr_nan_list[0]))
    for g in np.arange(1,G):
        Xtr_nan = np.vstack((Xtr_nan, Xtr_nan_list[g]))
        ytr = np.hstack((ytr, np.repeat(g, len(Xtr_nan_list[g]))))

    scaler = StandardScaler()
    scaler.fit(Xtr_nan)
    Xtr_nan = scaler.transform(Xtr_nan)
    Xtrain = scaler.transform(Xtrain)
    for g in range(G):
      Xtr_nan_list[g] = scaler.transform(Xtr_nan_list[g])

    mus = [np.mean(Xtrain[ytrain==g,:], axis=0) for g in np.arange(G)]
    mus = np.asarray(mus) # each row is a mean of a class
    S = [(sum(ytrain==g)-1)*np.cov(Xtrain[ytrain==g,:],rowvar =False) 
             for g in np.arange(G)]
    S = np.asarray(S)/len(ytrain)

    # percentage of missing values
    per_missing = np.mean(np.isnan(Xtr_nan))
    # MLEs approach
    start = time.time()
    Xtr_mice = IterativeImputer(max_iter=10).fit(Xtr_nan).transform(Xtr_nan)
    mus_mice = np.asarray([np.mean(Xtr_mice[ytrain==g,:], axis=0
                                   ) for g in np.arange(G)])
    S_mice = np.asarray([(sum(ytrain==g)-1)*np.cov(Xtr_mice[ytrain==g,:], rowvar =False) 
             for g in np.arange(G)])
    S_mice = S_mice/len(ytrain)
    mice_err = err(mus, S, mus_mice, S_mice)
    mice_time = time.time()-start  

    return mice_err, mice_time, per_missing

In [8]:
def compute_err_MLE(Xtrain, ytrain, n, p, G):      
    # make NAs
    Xtr_nan_list = make_nan_list(Xtrain,ytrain,G, n, p)
    # make NA data
    # since making function changes the order of observation
    # we need to generate new ytr from Xtr_nan    
    Xtr_nan, ytr = Xtr_nan_list[0], np.repeat(0, len(Xtr_nan_list[0]))
    for g in np.arange(1,G):
        Xtr_nan = np.vstack((Xtr_nan, Xtr_nan_list[g]))
        ytr = np.hstack((ytr, np.repeat(g, len(Xtr_nan_list[g]))))

    scaler = StandardScaler()
    scaler.fit(Xtr_nan)
    Xtr_nan = scaler.transform(Xtr_nan)
    Xtrain = scaler.transform(Xtrain)
    for g in range(G):
      Xtr_nan_list[g] = scaler.transform(Xtr_nan_list[g])

    mus = [np.mean(Xtrain[ytrain==g,:], axis=0) for g in np.arange(G)]
    mus = np.asarray(mus) # each row is a mean of a class
    S = [(sum(ytrain==g)-1)*np.cov(Xtrain[ytrain==g,:],rowvar =False) 
             for g in np.arange(G)]
    S = np.asarray(S)/len(ytrain)

    # percentage of missing values
    per_missing = np.mean(np.isnan(Xtr_nan))
    # MLEs approach
    start = time.time()
    mus_mle, S_mle = mle(Xtr_nan_list, n, p, G)
    mle_err = err(mus, S, mus_mle.T, S_mle)
    mle_time = time.time()-start    

    return mle_err, mle_time, per_missing

In [9]:
def compute_err_SOFT(Xtrain, ytrain, n, p, G):      
    # make NAs
    Xtr_nan_list = make_nan_list(Xtrain,ytrain,G, n, p)
    # make NA data
    # since making function changes the order of observation
    # we need to generate new ytr from Xtr_nan    
    Xtr_nan, ytr = Xtr_nan_list[0], np.repeat(0, len(Xtr_nan_list[0]))
    for g in np.arange(1,G):
        Xtr_nan = np.vstack((Xtr_nan, Xtr_nan_list[g]))
        ytr = np.hstack((ytr, np.repeat(g, len(Xtr_nan_list[g]))))

    scaler = StandardScaler()
    scaler.fit(Xtr_nan)
    Xtr_nan = scaler.transform(Xtr_nan)
    Xtrain = scaler.transform(Xtrain)
    for g in range(G):
      Xtr_nan_list[g] = scaler.transform(Xtr_nan_list[g])

    mus = [np.mean(Xtrain[ytrain==g,:], axis=0) for g in np.arange(G)]
    mus = np.asarray(mus) # each row is a mean of a class
    S = [(sum(ytrain==g)-1)*np.cov(Xtrain[ytrain==g,:],rowvar =False) 
             for g in np.arange(G)]
    S = np.asarray(S)/len(ytrain)

    # percentage of missing values
    per_missing = np.mean(np.isnan(Xtr_nan))

    start = time.time()
    Xtr_softimpute = SoftImpute(max_iters = 100).fit_transform(Xtr_nan)
    mus_softimpute = np.asarray([np.mean(Xtr_softimpute[ytrain==g,:], axis=0
                                         ) for g in np.arange(G)])
    S_softimpute = np.asarray([(sum(ytrain==g)-1)*np.cov(Xtr_softimpute[ytrain==g,:], rowvar =False) 
             for g in np.arange(G)])
    S_softimpute = S_softimpute/len(ytrain)
    softimpute_err =  err(mus, S, mus_softimpute, S_softimpute)
    softimpute_time = time.time()-start    

    return softimpute_err, softimpute_time, per_missing

# Import Fashion MNIST

In [10]:
import tensorflow as tf
fashion_mnist = tf.keras.datasets.fashion_mnist
(Xtrain, ytrain), (Xtest, ytest) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [11]:
Xtrain = Xtrain.astype(float).reshape((60000,784))

In [12]:
# convert the test set to NumPy arrays and flatten the data
Xtest = Xtest.astype(float).reshape((10000,784))

In [13]:
X = np.vstack((Xtrain, Xtest))
y = np.hstack((ytrain, ytest))

In [14]:
# number of sample per class in training data
ng = np.asarray([sum(y==i) for i in np.arange(10)])
ng

array([7000, 7000, 7000, 7000, 7000, 7000, 7000, 7000, 7000, 7000])

### 20%

In [15]:
n = np.hstack((ng.reshape((-1,1)), np.tile([5000,4500,4200, 4000],
                                 10).reshape((10,-1))))
p = np.array([380,450,500, 520,784])   
missing_rate(X, y, n, p, 10)

0.20280612244897958

In [16]:
compute_err_MLE(X, y, n, p, 10)

(0.00010450153060820514, 2.543551206588745, 0.20280612244897958)

In [17]:
compute_err_SOFT(X, y, n, p, 10)

[SoftImpute] Max Singular Value of X_init = 3030.780817
[SoftImpute] Iter 1: observed MAE=0.122046 rank=503
[SoftImpute] Iter 2: observed MAE=0.122131 rank=498
[SoftImpute] Iter 3: observed MAE=0.122177 rank=496
[SoftImpute] Iter 4: observed MAE=0.122196 rank=495
[SoftImpute] Iter 5: observed MAE=0.122200 rank=494
[SoftImpute] Iter 6: observed MAE=0.122198 rank=494
[SoftImpute] Iter 7: observed MAE=0.122192 rank=494
[SoftImpute] Iter 8: observed MAE=0.122184 rank=494
[SoftImpute] Iter 9: observed MAE=0.122175 rank=494
[SoftImpute] Iter 10: observed MAE=0.122165 rank=494
[SoftImpute] Iter 11: observed MAE=0.122155 rank=494
[SoftImpute] Iter 12: observed MAE=0.122146 rank=494
[SoftImpute] Iter 13: observed MAE=0.122136 rank=494
[SoftImpute] Iter 14: observed MAE=0.122127 rank=494
[SoftImpute] Iter 15: observed MAE=0.122118 rank=494
[SoftImpute] Iter 16: observed MAE=0.122110 rank=494
[SoftImpute] Iter 17: observed MAE=0.122102 rank=494
[SoftImpute] Iter 18: observed MAE=0.122094 rank=494

(0.006429311626715689, 944.2510235309601, 0.20280612244897958)

### 30%

In [18]:
    n = np.hstack((ng.reshape((-1,1)), np.tile([4000,3500,3000, 2800],
                                 10).reshape((10,-1))))
    p = np.array([350,450,500, 520,784])   
    missing_rate(X, y, n, p, 10)

0.30317055393586007

In [19]:
compute_err_MLE(X, y, n, p, 10)

(0.00012231061717755495, 2.11018705368042, 0.30317055393586007)

In [20]:
compute_err_SOFT(X, y, n, p, 10)

[SoftImpute] Max Singular Value of X_init = 2853.075334
[SoftImpute] Iter 1: observed MAE=0.120687 rank=494
[SoftImpute] Iter 2: observed MAE=0.120748 rank=486
[SoftImpute] Iter 3: observed MAE=0.120792 rank=482
[SoftImpute] Iter 4: observed MAE=0.120814 rank=481
[SoftImpute] Iter 5: observed MAE=0.120822 rank=481
[SoftImpute] Iter 6: observed MAE=0.120823 rank=480
[SoftImpute] Iter 7: observed MAE=0.120819 rank=480
[SoftImpute] Iter 8: observed MAE=0.120812 rank=480
[SoftImpute] Iter 9: observed MAE=0.120804 rank=480
[SoftImpute] Iter 10: observed MAE=0.120796 rank=480
[SoftImpute] Iter 11: observed MAE=0.120787 rank=480
[SoftImpute] Iter 12: observed MAE=0.120778 rank=480
[SoftImpute] Iter 13: observed MAE=0.120769 rank=480
[SoftImpute] Iter 14: observed MAE=0.120760 rank=480
[SoftImpute] Iter 15: observed MAE=0.120751 rank=480
[SoftImpute] Iter 16: observed MAE=0.120743 rank=480
[SoftImpute] Iter 17: observed MAE=0.120735 rank=480
[SoftImpute] Iter 18: observed MAE=0.120728 rank=480

(0.006425510848236525, 984.7857704162598, 0.30317055393586007)

## 40%

In [21]:
n = np.hstack((ng.reshape((-1,1)), np.tile([4500,3490,3350, 3100],
                                 10).reshape((10,-1))))
p = np.array([180,250,400, 450,784])   
missing_rate(X, y, n, p, 10)

0.398432944606414

In [22]:
compute_err_MLE(X, y, n, p, 10)

(0.00014340962350528415, 1.6470777988433838, 0.398432944606414)

In [23]:
compute_err_SOFT(X, y, n, p, 10)

[SoftImpute] Max Singular Value of X_init = 2561.391930
[SoftImpute] Iter 1: observed MAE=0.117163 rank=503
[SoftImpute] Iter 2: observed MAE=0.117282 rank=497
[SoftImpute] Iter 3: observed MAE=0.117352 rank=495
[SoftImpute] Iter 4: observed MAE=0.117389 rank=495
[SoftImpute] Iter 5: observed MAE=0.117407 rank=494
[SoftImpute] Iter 6: observed MAE=0.117414 rank=494
[SoftImpute] Iter 7: observed MAE=0.117414 rank=493
[SoftImpute] Iter 8: observed MAE=0.117411 rank=493
[SoftImpute] Iter 9: observed MAE=0.117406 rank=493
[SoftImpute] Iter 10: observed MAE=0.117399 rank=493
[SoftImpute] Iter 11: observed MAE=0.117392 rank=493
[SoftImpute] Iter 12: observed MAE=0.117384 rank=493
[SoftImpute] Iter 13: observed MAE=0.117376 rank=493
[SoftImpute] Iter 14: observed MAE=0.117368 rank=493
[SoftImpute] Iter 15: observed MAE=0.117360 rank=493
[SoftImpute] Iter 16: observed MAE=0.117352 rank=493
[SoftImpute] Iter 17: observed MAE=0.117344 rank=493
[SoftImpute] Iter 18: observed MAE=0.117336 rank=493

(0.006417907480551935, 1006.9230966567993, 0.398432944606414)

## 50%

In [24]:
n = np.hstack((ng.reshape((-1,1)), np.tile([3500,3290,3100, 3000],
                                 10).reshape((10,-1))))
p = np.array([90,110,200, 250,784])   
missing_rate(X, y, n, p, 10)

0.4983418367346939

In [25]:
compute_err_MLE(X, y, n, p, 10)

(0.00016006886880623065, 1.0960605144500732, 0.4983418367346939)

In [26]:
compute_err_SOFT(X, y, n, p, 10)

[SoftImpute] Max Singular Value of X_init = 2309.961374
[SoftImpute] Iter 1: observed MAE=0.115912 rank=502
[SoftImpute] Iter 2: observed MAE=0.116007 rank=500
[SoftImpute] Iter 3: observed MAE=0.116066 rank=498
[SoftImpute] Iter 4: observed MAE=0.116103 rank=498
[SoftImpute] Iter 5: observed MAE=0.116125 rank=498
[SoftImpute] Iter 6: observed MAE=0.116139 rank=498
[SoftImpute] Iter 7: observed MAE=0.116149 rank=498
[SoftImpute] Iter 8: observed MAE=0.116154 rank=498
[SoftImpute] Iter 9: observed MAE=0.116158 rank=498
[SoftImpute] Iter 10: observed MAE=0.116160 rank=498
[SoftImpute] Iter 11: observed MAE=0.116161 rank=498
[SoftImpute] Iter 12: observed MAE=0.116161 rank=498
[SoftImpute] Iter 13: observed MAE=0.116161 rank=498
[SoftImpute] Iter 14: observed MAE=0.116160 rank=498
[SoftImpute] Iter 15: observed MAE=0.116159 rank=498
[SoftImpute] Iter 16: observed MAE=0.116158 rank=498
[SoftImpute] Iter 17: observed MAE=0.116157 rank=498
[SoftImpute] Iter 18: observed MAE=0.116155 rank=498

(0.006415598705637204, 1017.2588453292847, 0.4983418367346939)